In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import to_date

# Set JAVA_HOME (penting untuk Mac Intel)
os.environ["JAVA_HOME"] = "/usr/local/opt/openjdk@11"

# Inisialisasi Spark
import findspark
findspark.init()

spark = SparkSession.builder.appName("Silver_ETL_Penjualan").getOrCreate()

# Definisikan schema untuk memastikan tipe data sesuai (biar siap ke data warehouse)
schema = StructType([
    StructField("id_transaksi", StringType(), True),
    StructField("id_produk", StringType(), True),
    StructField("jumlah", FloatType(), True),
    StructField("harga_satuan", FloatType(), True),
    StructField("tanggal_transaksi", StringType(), True)  # sementara masih string
])

# Load data mentah dengan schema yang sudah ditentukan
df_raw = spark.read.option("header", True).schema(schema).csv("penjualan_2025-07-03.csv")

# Convert tanggal_transaksi jadi format date (YYYY-MM-DD)
df_silver = df_raw.withColumn("tanggal_transaksi", to_date("tanggal_transaksi", "dd-MM-yyyy"))

# Preview hasil silver
#df_silver.printSchema()
df_silver.show()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/07/07 08:16:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/07 08:16:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
+------------+---------+------+------------+-----------------+
|id_transaksi|id_produk|jumlah|harga_satuan|tanggal_transaksi|
+------------+---------+------+------------+-----------------+
|      TRX001|   PROD-A|   3.0|     86653.0|       2025-07-03|
|      TRX002|   PROD-B|   8.0|     27843.0|       2025-07-03|
|      TRX003|   PROD-C|   8.0|     73726.0|       2025-07-03|
|      TRX004|   PROD-A|  null|     39648.0|       2025-07-03|
|      TRX005|   PROD-B|   4.0|     18049.0|       2025-07-03|
|      TRX006|   prod-a|  10.0|     38823.0|       2025-07-03|
|      TRX007|   PROD-A|   7.0|     51941.0|       2025-07-03|
|      TRX008|   PROD-B|   2.0|     70892.0|       2025-07-03|
|      TRX009|   PROD-C|   5.0|     62747.0|       2025-07-03|
| 

In [5]:
from pyspark.sql.functions import col, upper
from pyspark.sql.types import IntegerType

# Transformasi Bronze dari df_silver
df_bronze = (
    df_silver
    # Seragamkan id_produk jadi huruf kapital
    .withColumn("id_produk", upper(col("id_produk")))
    # Ubah jumlah ke IntegerType
    .withColumn("jumlah", col("jumlah").cast(IntegerType()))
    # Ubah harga_satuan ke IntegerType
    .withColumn("harga_satuan", col("harga_satuan").cast(IntegerType()))
    # Tambahkan kolom total_harga = jumlah * harga_satuan
    .withColumn("total_harga", (col("jumlah") * col("harga_satuan")).cast(IntegerType()))
    # Isi semua kolom null (string -> "", angka -> 0, tanggal -> default)
    .fillna({
        "id_transaksi": "",
        "id_produk": "",
        "jumlah": 0,
        "harga_satuan": 0,
        "total_harga": 0,
        "tanggal_transaksi": "1970-01-01"
    })
)

# Urutkan kolom biar tanggal_transaksi paling kanan
df_bronze = df_bronze.select(
    "id_transaksi",
    "id_produk",
    "jumlah",
    "harga_satuan",
    "total_harga",
    "tanggal_transaksi"
)

# Tampilkan schema dan isi data
#df_bronze.printSchema()
df_bronze.show()


+------------+---------+------+------------+-----------+-----------------+
|id_transaksi|id_produk|jumlah|harga_satuan|total_harga|tanggal_transaksi|
+------------+---------+------+------------+-----------+-----------------+
|      TRX001|   PROD-A|     3|       86653|     259959|       2025-07-03|
|      TRX002|   PROD-B|     8|       27843|     222744|       2025-07-03|
|      TRX003|   PROD-C|     8|       73726|     589808|       2025-07-03|
|      TRX004|   PROD-A|     0|       39648|          0|       2025-07-03|
|      TRX005|   PROD-B|     4|       18049|      72196|       2025-07-03|
|      TRX006|   PROD-A|    10|       38823|     388230|       2025-07-03|
|      TRX007|   PROD-A|     7|       51941|     363587|       2025-07-03|
|      TRX008|   PROD-B|     2|       70892|     141784|       2025-07-03|
|      TRX009|   PROD-C|     5|       62747|     313735|       2025-07-03|
|      TRX010|   PROD-A|     1|       61061|      61061|       2025-07-03|
|      TRX011|   PROD-B| 